## Importing packages and loading data

In [20]:
# Import necesscary packages
import pandas as pd
import numpy as np
import time

In [26]:
Orders = pd.read_excel("OrderList.xlsx", header=6, index_col=0)
Orders

,Position 1,Position 2,Position 3,Position 4,Position 5
Order No.,,,,,
1,50,30,0,0,0
2,49,18,76,0,0
3,72,52,51,41,35
4,50,4,0,0,0
5,76,19,26,80,6
...,...,...,...,...,...
1996,60,46,35,0,0
1997,8,43,70,77,31
1998,46,0,0,0,0


In [25]:
Allocations = pd.read_csv("CurrentAllocation (table).csv", index_col=0, header=3)
Allocations

,Product Group
Shelf,
1,45
2,79
3,39
4,68
5,73
...,...
92,0
93,0
94,0


In [5]:
DistanceMatrix = pd.read_excel("DistanceMatrix.xlsx", sheet_name="DistanceMatrix Squares", index_col=0, header=2)
DistanceMatrix

,1,2,3,4,5,6,7,8,9,10,...,88,89,90,91,92,93,94,95,96,Packaging
1,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,2
2,1,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,27,3
3,2,1,0,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,27,26,4
4,3,2,1,0,1,2,3,4,5,6,...,23,24,25,26,27,28,27,26,25,5
5,4,3,2,1,0,1,2,3,4,5,...,24,25,26,27,28,27,26,25,24,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,25,26,27,28,27,26,25,24,23,22,...,5,4,3,2,1,0,1,2,3,23
94,26,27,28,27,26,25,24,23,22,21,...,6,5,4,3,2,1,0,1,2,24
95,27,28,27,26,25,24,23,22,21,20,...,7,6,5,4,3,2,1,0,1,25
96,28,27,26,25,24,23,22,21,20,19,...,8,7,6,5,4,3,2,1,0,26


# Q1

## Distance function (square) using default order 

In [35]:
def distance_fun_default(Allocation, Order):
    total_square = 0
    order_num = Order.shape[0]
    order_size = Order.shape[1]

    # Loop over each order
    for i in range(order_num):
        order = Order.iloc[i]
        current_shelf = "Packaging"

        # Loop over each product group in each order
        for j in range(order_size):
            if order.iloc[j] != 0:
                # Find all shelves where the current product group is located
                product_shelves = Allocation.index[Allocation['Product Group'] == order.iloc[j]].tolist()

                # Go to each shelf in the order they appear
                for k in product_shelves:
                    current_distance = DistanceMatrix.at[current_shelf, k]
                    total_square += current_distance
                    current_shelf = k 

        # Add the distance back to the packaging area
        total_square += DistanceMatrix.at[current_shelf, "Packaging"]
        
    return(total_square)

In [34]:
start_time = time.time()
print(distance_fun_default(Allocations, Orders))
end_time = time.time()
runtime = end_time - start_time
print(f"The runtime of the distance function using default order was: {runtime} seconds")

125080
The runtime of the distance function using default order was: 0.9440076351165771 seconds


## Distance function (square) using Greedy method 

In [30]:
def distance_fun_greedy(Allocation, Order):
    total_square = 0
    order_num = Order.shape[0]
    order_size = Order.shape[1]

    # Loop over each order
    for i in range(order_num):
        order = Order.iloc[i]
        current_shelf = "Packaging"
        
        # Count the number of products that need to be picked
        num_goods = np.count_nonzero(order)

        # Create a list to track the products that have already been picked
        visited_goods = []
        
        # Continue picking goods until all products in the order have been visited
        while len(visited_goods) < num_goods:
            within_order_distances = {}

            # Loop over each product group in each order
            for j in range(order_size):
                if order.iloc[j] != 0 and order.iloc[j] not in visited_goods:
                    # Find all shelves where the current product group is located
                    within_good_shelves = Allocation.index[Allocation['Product Group'] == order.iloc[j]].tolist()
                    within_good_distances = {} 

                    # Calculate the distance from the current shelf to each next shelf
                    for k in within_good_shelves:
                        within_good_distances[k] = DistanceMatrix[current_shelf][k]
                    
                    # Choose the next shelf with the minimum distance from the current shelf
                    least_within_good_shelf = min(within_good_distances, key=within_good_distances.get)
                    least_within_good_distance = within_good_distances[least_within_good_shelf]

                    # Add the closest shelf and its distance to the within_order_distances dictionary
                    within_order_distances[least_within_good_shelf] = least_within_good_distance

            least_within_order_shelf = min(within_order_distances, key=within_order_distances.get)
            next_shelf = least_within_order_shelf
            least_within_order_distance = within_order_distances[least_within_order_shelf]
            next_good = Allocation._get_value(next_shelf, "Product Group")
            visited_goods.append(next_good)
            current_shelf = next_shelf
            total_square += least_within_order_distance
        total_square += DistanceMatrix.at[current_shelf, "Packaging"]
        
    return(total_square)

In [32]:
start_time = time.time()
print(distance_fun_greedy(Allocations, Orders))
end_time = time.time()
runtime = end_time - start_time
print(f"The runtime of the distance function using greedy method was: {runtime} seconds")

96878
The runtime of the distance function using greedy method was: 2.8658576011657715 seconds


# Q2

##  Random allocate generator

In [17]:
def allocation_gen():
    # Create a DataFrame with integers from 1 to 90
    df = pd.DataFrame({'Product Group': range(1, 91)})

    # Shuffle the order of integers randomly
    df = df.sample(frac=1).reset_index(drop=True)

    # Generate 6 random unique integers from 1 to 90
    additional_integers = np.random.choice(np.arange(1, 91), size=6, replace=False)

    # Create a DataFrame for additional integers
    additional_df = pd.DataFrame({'Product Group': additional_integers})

    # Concatenate the additional integers to the existing DataFrame
    df = pd.concat([df, additional_df], ignore_index=True)

    df.index = np.arange(1, 97)
    df.index.name = 'shelf'

    return(df)

## Random select method

In [18]:
# Randomly generte allocations, select the best one
np.random.seed(0)
Allo_init = allocation_gen()
distance_init = distance_fun_greedy(Allo_init)
for i in range(20):
    Allo = allocation_gen()
    distance = distance_fun_greedy(Allo)
    if distance <= distance_init:
        distance_init = distance
        Allo_init = Allo
Allo_random_select = Allo_init
distance_random_select = distance_init
print(Allo_random_select)
print(distance_random_select)

       Product Group
shelf               
1                 41
2                 56
3                 77
4                 12
5                 64
...              ...
92                 6
93                18
94                57
95                23
96                41

[96 rows x 1 columns]
89950


# Q3

## Local Search Heuristics 

In [19]:
# Local search for the given allocation
for i in range(95):
    Allo = Allocation
    Distance_init = distance_fun_greedy(Allo)

    # Swap values that neighboors
    index1 = i
    index2 = i + 1

    # Create a new DataFrame to store the swapped values
    Allo_next = Allo.copy()

    # Get the values at the specified indices
    value1 = Allo.iloc[index1, 0]
    value2 = Allo.iloc[index2, 0]

    # Swap the values
    Allo_next.iloc[index1, 0] = value2
    Allo_next.iloc[index2, 0] = value1

    # Find the allocation and distance 
    Allo_next = Allo_next
    Distance_next = distance_fun_greedy(Allo_next)

    if Distance_next <= Distance_init:
        Distance_init = Distance_next
        Allo = Allo_next
print(Allo)
print(Distance_init)

       Product Group
Shelf               
1                 45
2                 79
3                 39
4                 68
5                 73
...              ...
92                 0
93                 0
94                 0
95                 0
96                 0

[96 rows x 1 columns]
96878


In [28]:
# Local search for the allocation provided by random select
for i in range(95):
    Allo = Allo_random_select
    Distance_init = distance_fun_greedy(Allo)

    # Swap values that neighboors
    index1 = i
    index2 = i + 1

    # Create a new DataFrame to store the swapped values
    Allo_next = Allo.copy()

    # Get the values at the specified indices
    value1 = Allo.iloc[index1, 0]
    value2 = Allo.iloc[index2, 0]

    # Swap the values
    Allo_next.iloc[index1, 0] = value2
    Allo_next.iloc[index2, 0] = value1

    # Find the allocation and distance 
    Allo_next = Allo_next
    Distance_next = distance_fun_greedy(Allo_next)

    if Distance_next <= Distance_init:
        Distance_init = Distance_next
        Allo = Allo_next
print(Allo)
print(Distance_init)

/tmp/ipykernel_2194/613294864.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2194/613294864.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist()


       Product Group
shelf               
1                 50
2                 19
3                 34
4                 80
5                 71
...              ...
92                14
93                54
94                42
95                31
96                67

[96 rows x 1 columns]
93378


## Q4

## Constructing the distance matrix for our shelves layout

In [29]:
DistanceMatrix2 = pd.read_excel("DistanceMatrix2.xlsx", sheet_name="DistanceMatrix Squares", index_col=0, header=2)
DistanceMatrix2

,1,2,3,4,5,6,7,8,9,10,...,88,89,90,91,92,93,94,95,96,Packaging
1,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,4,2
2,1,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,5,3
3,2,1,0,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,27,6,4
4,3,2,1,0,1,2,3,4,5,6,...,23,24,25,26,27,28,27,26,7,5
5,4,3,2,1,0,1,2,3,4,5,...,24,25,26,27,28,27,26,25,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,25,26,27,28,27,26,25,24,23,22,...,5,4,3,2,1,0,1,2,21,23
94,26,27,28,27,26,25,24,23,22,21,...,6,5,4,3,2,1,0,1,22,24
95,27,28,27,26,25,24,23,22,21,20,...,7,6,5,4,3,2,1,0,23,25
96,4,5,6,7,8,9,10,11,12,13,...,16,17,18,19,20,21,22,23,0,2


## Distance function in default order wiht new layout

In [30]:
def distance_fun_default2(allo):
    distance = 0
    for i in range(2000):
        order = Orders.iloc[i]
        current_shelf = "Packaging"
        for j in range(5):
            if order.iloc[j] != 0:
                next_shelves = allo.index[allo['Product Group'] == order.iloc[j]].tolist()
                current_distances = {}
                for k in next_shelves:
                    current_distances[k] = DistanceMatrix2.at[current_shelf, k]
                next_shelf = min(current_distances, key=current_distances.get)
                current_distance = current_distances[next_shelf]
                distance += current_distance
            current_shelf = next_shelf
        distance += DistanceMatrix2.at[current_shelf, "Packaging"]
    return(distance)

In [37]:
print(distance_fun_default(Allocation))
print(distance_fun_default2(Allocation))

/tmp/ipykernel_2194/2969029687.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0:
/tmp/ipykernel_2194/2969029687.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_shelves = allo.index[allo['Product Group'] == order[j]].tolist()


125080
120150


# Distance function in greedy with new layout

In [32]:
def distance_fun_greedy2(allo):
    distance = 0
    for i in range(2000):
        order = Orders.iloc[i]
        num_goods = np.count_nonzero(order)
        current_shelf = "Packaging"

        visited_goods = []
        
        while len(visited_goods) < num_goods:
            within_order_distances = {}
            for j in range(5):
                if order.iloc[j] != 0 and order.iloc[j] not in visited_goods:
                    within_good_shelves = allo.index[allo['Product Group'] == order.iloc[j]].tolist()
                    within_good_distances = {} 
                    for k in within_good_shelves:
                        within_good_distances[k] = DistanceMatrix2[current_shelf][k]
                    least_within_good_shelf = min(within_good_distances, key=within_good_distances.get)
                    least_within_good_distance = within_good_distances[least_within_good_shelf]

                    within_order_distances[least_within_good_shelf] = least_within_good_distance

            least_within_order_shelf = min(within_order_distances, key=within_order_distances.get)
            next_shelf = least_within_order_shelf
            least_within_order_distance = within_order_distances[least_within_order_shelf]
            next_good = allo._get_value(next_shelf, "Product Group")
            visited_goods.append(next_good)
            current_shelf = next_shelf
            distance += least_within_order_distance
        distance += DistanceMatrix2.at[current_shelf, "Packaging"]
        
    return(distance)

In [35]:
print(distance_fun_greedy(Allocation))
print(distance_fun_greedy2(Allocation))

/tmp/ipykernel_2194/613294864.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2194/613294864.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist()


96878
93718


## Create a initial allocation by using random selection

In [40]:
# Randomly generte allocations, select the best one
np.random.seed(0)
Allo_init = allocation_gen()
distance_init = distance_fun_greedy2(Allo_init)
for i in range(20):
    Allo = allocation_gen()
    distance = distance_fun_greedy2(Allo)
    if distance <= distance_init:
        distance_init = distance
        Allo_init = Allo
Allo_random_select = Allo_init
distance_random_select = distance_init
print(Allo_random_select)
print(distance_random_select)

/tmp/ipykernel_2194/3015275244.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2194/3015275244.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist()


       Product Group
shelf               
1                 50
2                 19
3                 34
4                 80
5                 71
...              ...
92                14
93                54
94                42
95                67
96                31

[96 rows x 1 columns]
90510


### Local search heuristics with new distance matrix

In [39]:
# Local search with new distance matrix for the given allocation
for i in range(95):
    Allo = Allocation
    Distance_init = distance_fun_greedy2(Allo)

    # Swap values that neighboors
    index1 = i
    index2 = i + 1

    # Create a new DataFrame to store the swapped values
    Allo_next = Allo.copy()

    # Get the values at the specified indices
    value1 = Allo.iloc[index1, 0]
    value2 = Allo.iloc[index2, 0]

    # Swap the values
    Allo_next.iloc[index1, 0] = value2
    Allo_next.iloc[index2, 0] = value1

    # Find the allocation and distance 
    Allo_next = Allo_next
    Distance_next = distance_fun_greedy2(Allo_next)

    if Distance_next <= Distance_init:
        Distance_init = Distance_next
        Allo = Allo_next
print(Allo)
print(Distance_init)

/tmp/ipykernel_2194/3015275244.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2194/3015275244.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist()


       Product Group
Shelf               
1                 45
2                 79
3                 39
4                 68
5                 73
...              ...
92                 0
93                 0
94                 0
95                 0
96                 0

[96 rows x 1 columns]
93718


In [41]:
# Local search with new distance matrix for the allocation provided by random select
for i in range(95):
    Allo = Allo_random_select
    Distance_init = distance_fun_greedy2(Allo)

    # Swap values that neighboors
    index1 = i
    index2 = i + 1

    # Create a new DataFrame to store the swapped values
    Allo_next = Allo.copy()

    # Get the values at the specified indices
    value1 = Allo.iloc[index1, 0]
    value2 = Allo.iloc[index2, 0]

    # Swap the values
    Allo_next.iloc[index1, 0] = value2
    Allo_next.iloc[index2, 0] = value1

    # Find the allocation and distance 
    Allo_next = Allo_next
    Distance_next = distance_fun_greedy2(Allo_next)

    if Distance_next <= Distance_init:
        Distance_init = Distance_next
        Allo = Allo_next
print(Allo)
print(Distance_init)

/tmp/ipykernel_2194/3015275244.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2194/3015275244.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist()


       Product Group
shelf               
1                 50
2                 19
3                 34
4                 80
5                 71
...              ...
92                14
93                54
94                42
95                67
96                31

[96 rows x 1 columns]
90510
